# Bases de datos

In [1]:
import pandas as pd
#import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import openpyxl
import pandas.io.formats.excel 
import time
%matplotlib inline
#pd.set_option('display.notebook_repr_html', True)
#pd.set_option('display.max_columns', 11)
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.width', 78)
pd.set_option('precision', 3)
pd.options.display.float_format = '{:.5f}'.format


In [2]:
data=pd.read_excel('annual_generation_state.xls','TOTALES',index_col=0)
data=data.loc[data['GENERATION (Megawatthours)']>0,:]

In [3]:
years=np.unique(data.index)

In [4]:
states=np.unique(data['STATE'])
states

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'US-TOTAL', 'US-Total', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'],
      dtype=object)

In [5]:
energies=np.unique(data['ENERGY SOURCE'])
energies

array(['Coal', 'Geothermal', 'Hydroelectric Conventional', 'Natural Gas',
       'Nuclear', 'Other', 'Other Biomass', 'Other Gases', 'Petroleum',
       'Pumped Storage', 'Solar Thermal and Photovoltaic', 'Wind',
       'Wood and Wood Derived Fuels'], dtype=object)

In [6]:
total_by_states=list(map(lambda s:list(map(lambda y:sum(data.loc[data['STATE']==states[s],'GENERATION (Megawatthours)'].loc[data.loc[data['STATE']==states[s],:].index==years[y]]),range(len(years)))),range(len(states))))

In [7]:
data_by_states=list(map(lambda e:data.loc[data['STATE']==states[e],:],range(len(states))))

In [8]:
data_by_states=list(map(lambda s:list(map(lambda y:(data.loc[data['STATE']==states[s],:].loc[data.loc[data['STATE']==states[s],:].index==years[y]]),range(len(years)))),range(len(states))))

In [9]:
#state,year
proportions_by_states=list(map(lambda s:list(map(lambda y:pd.concat([data_by_states[s][y],data_by_states[s][y]['GENERATION (Megawatthours)']/total_by_states[s][y]],axis=1),range(len(years)))),range(len(states))))

In [10]:
#state,type of energy
df=list(map(lambda s:list(map(lambda e:pd.DataFrame(np.reshape((np.column_stack(list(map(lambda y:pd.DataFrame(np.ones([1,4])*.0001) if (proportions_by_states[s][y].loc[proportions_by_states[s][y]['ENERGY SOURCE']==energies[e],:]).empty else proportions_by_states[s][y].loc[proportions_by_states[s][y]['ENERGY SOURCE']==energies[e],:],range(len(years)))))),(28,4))),range(len(energies)))),range(len(states))))

In [11]:
#state,type of energy
df_with_ret=list(map(lambda s:list(map(lambda e:pd.concat([df[s][e],(df[s][e][3]/df[s][e][3].shift(1)-1)*100],axis=1),range(len(energies)))),range(len(states))))

In [12]:
df_with_rets=list(map(lambda s:list(map(lambda e:pd.concat([df_with_ret[s][e],(df[s][e][2]/df[s][e][2].shift(1)-1)*100],axis=1),range(len(energies)))),range(len(states))))

In [13]:
df_with_rets_dates=list(map(lambda s:list(map(lambda e:df_with_rets[s][e].set_index(years),range(len(energies)))),range(len(states))))

In [14]:
#El codio queda comentado para que cada vez que se corra todo no afecte el primer archivo ya generado

#doc=openpyxl.load_workbook('database.xlsx')
#hoja=doc.get_sheet_by_name('Hoja1')
#list(map(lambda s:list(map(lambda e:list(map(lambda f: hoja.append([""]) if pd.DataFrame.equals(df_with_rets_dates[s][e],df_with_rets_dates[0][5]) else hoja.append([df_with_rets_dates[s][e].index[f],df_with_rets_dates[s][e].iloc[f,0],df_with_rets_dates[s][e].iloc[f,1],df_with_rets_dates[s][e].iloc[f,2],df_with_rets_dates[s][e].iloc[f,3],df_with_rets_dates[s][e].iloc[f,4],df_with_rets_dates[s][e].iloc[f,5]]),range(28))),range(len(energies)))),range(len(states))))
#doc.save('database.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  


In [57]:
df_total_by_state=pd.DataFrame(np.reshape(total_by_states,[53,28]).T).set_index(years)
df_total_by_state.columns=states
df_total_by_state=pd.concat([df_total_by_state,df_total_by_state/df_total_by_state.shift(1)-1],axis=1)
df_total_by_state

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,TX,US-TOTAL,US-Total,UT,VA,VT,WA,WI,WV,WY
1990,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1991,-0.03745,0.11468,0.03968,0.07054,-0.04124,-0.00742,-0.23066,-0.50196,-0.00043,0.06018,...,0.01694,0.01217,nan,-0.06319,0.04436,0.04335,0.00783,0.03324,-0.07689,-0.01693
1992,-0.01696,0.07262,-0.02227,0.04985,0.11458,0.02955,0.08030,-0.58851,-0.10894,0.03128,...,0.01384,0.00316,nan,0.08953,0.02242,-0.09611,-0.15931,-0.00939,0.02365,0.08080
1993,0.07981,0.04308,0.01893,-0.02953,0.05549,0.02837,0.12589,1.54696,0.29114,0.05673,...,0.03237,0.03665,nan,0.02470,0.08770,-0.04648,0.01750,0.02751,-0.00547,-0.04008
1994,0.04286,0.01437,0.03787,0.05061,-0.00427,0.04123,-0.05199,0.45529,0.02358,0.04089,...,0.02500,0.01552,nan,0.03343,0.01247,0.20959,0.00671,0.04221,0.08937,0.05472
1995,0.00999,0.04392,-0.00143,-0.02994,-0.02540,-0.00097,0.00981,-0.31136,-0.02139,0.05860,...,0.03404,0.03239,nan,-0.06789,0.01550,-0.08135,0.15365,0.02987,-0.01096,-0.06397
1996,0.01706,0.15480,0.09677,0.02685,-0.03417,0.04605,-0.35806,-0.41858,-0.01975,-0.00197,...,0.03562,0.02713,nan,0.00440,0.06211,0.03683,0.16107,0.00917,0.08328,0.03032
1997,0.00423,-0.01589,-0.01825,0.10007,-0.01407,0.01034,-0.11899,-0.35651,-0.18429,0.00695,...,0.02241,0.01420,nan,0.05342,0.02443,0.06176,0.02923,-0.05520,0.04911,-0.00151
1998,-0.05297,0.00268,0.01044,0.04148,0.09724,0.03291,0.10586,2.45275,-0.04487,0.12709,...,0.05650,0.03677,nan,0.03452,0.05442,-0.12926,-0.16350,0.09790,0.01413,0.09509
1999,-0.00455,0.00664,0.01946,0.02190,-0.00472,0.01645,0.45225,-0.05727,-0.00679,-0.01247,...,0.01020,0.02101,nan,0.02434,0.02694,0.15310,0.14610,0.03705,0.02015,-0.03781


In [ ]:
doc=openpyxl.load_workbook('database_Totals.xlsx')
hoja=doc.get_sheet_by_name('Hoja1')
hoja.append(df_total_by_state)
doc.save('database_Totals.xlsx')